## 데코레이터 소개

**데코레이터**는 자신이 감싸고 있는 함수가 호출되기 전과 후에 코드를 추가로 실행해준다.<br>
따라서 <u>감싸고 있는 함수의 입력 인자, 반환 값, 함수에서 발생한 오류에 접근할 수 있다.</u><br><br>
데코레이터를 함수에 적용할 때는 **@기호**를 사용한다.
> fibonacci(4)는 fibonacci(2)와 fibonacci(3)을 호출<br>
fibonacci(3)은 fibonacci(1)과 fibonacci(2)를 호출<br>
fibonacci(2)는 fibonacci(0)과 fibonacci(1)을 호출<br>
fibonacci(1)은 1, fibonacci(0)은 0

fibonacci함수가 호출 될 때 항상 전, 후에 데코레이터가 실행된다.
<br>
fibonacci(4)의 실행 순서: f(2) [ - f(0) - f(1)] - f(3) [ - f(1) - f(2) [ - f(0) - f(1)]]

In [22]:
# 데코레이터
def trace(func):
    def wrapper(*args, **kwargs):
        print('fibonacci 함수 시작: ', end='')
        print(*args)
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) -> {result!r}')
        print('fibonacci 함수 끝\n')
        return result
    return wrapper

@trace
def fibonacci(n):
    """Return n 번째 피보나치 수"""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

fibonacci(4)

fibonacci 함수 시작: 4
fibonacci 함수 시작: 2
fibonacci 함수 시작: 0
fibonacci((0,), {}) -> 0
fibonacci 함수 끝

fibonacci 함수 시작: 1
fibonacci((1,), {}) -> 1
fibonacci 함수 끝

fibonacci((2,), {}) -> 1
fibonacci 함수 끝

fibonacci 함수 시작: 3
fibonacci 함수 시작: 1
fibonacci((1,), {}) -> 1
fibonacci 함수 끝

fibonacci 함수 시작: 2
fibonacci 함수 시작: 0
fibonacci((0,), {}) -> 0
fibonacci 함수 끝

fibonacci 함수 시작: 1
fibonacci((1,), {}) -> 1
fibonacci 함수 끝

fibonacci((2,), {}) -> 1
fibonacci 함수 끝

fibonacci((3,), {}) -> 2
fibonacci 함수 끝

fibonacci((4,), {}) -> 3
fibonacci 함수 끝



3

'**@기호**를 사용하는 것은 이 함수(fibonacci)에 대해 데코레이터(trace)를 호출 한 후, 데코레이터가 반환한 결과(trace(fibonacci))를 원래 함수가 속해야 하는 영역에 원래 함수와 같은 이름으로 등록하는 것과 같다'의 의미.

In [4]:
def fibonacci2(n):
    """Return n 번째 피보나치 수"""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

fibonacci2 = trace(fibonacci2)
fibonacci2(4)

fibonacci 호출 전
fibonacci 호출 전
fibonacci 호출 전
fibonacci((0,), {}) -> 0
fibonacci 호출 후

fibonacci 호출 전
fibonacci((1,), {}) -> 1
fibonacci 호출 후

fibonacci((2,), {}) -> 1
fibonacci 호출 후

fibonacci 호출 전
fibonacci 호출 전
fibonacci((1,), {}) -> 1
fibonacci 호출 후

fibonacci 호출 전
fibonacci 호출 전
fibonacci((0,), {}) -> 0
fibonacci 호출 후

fibonacci 호출 전
fibonacci((1,), {}) -> 1
fibonacci 호출 후

fibonacci((2,), {}) -> 1
fibonacci 호출 후

fibonacci((3,), {}) -> 2
fibonacci 호출 후

fibonacci2((4,), {}) -> 3
fibonacci 호출 후



3

## 문제 발생
- 데코레이터가 반환하는 함수의 이름이 fibonacci가 아니게 된다.
- 이유: 데코레이터인 trace함수는 wrapper함수를 반환하므로, 데코레이터로 인해 wrapper함수가 모듈에 fibonacci라는 이름으로 등록된다.
- intropection을 하는 도구에서 문제가 된다.
> 인트로펙션(intropection): 실행 시점에 프로그램이 어떻게 실행되는지 관찰하는 것.

In [17]:
print(fibonacci)

<function trace.<locals>.wrapper at 0x000001440A27B940>


fibonacci함수의 독스트링이 출력되지 않으므로 쓸모가 없다.

In [19]:
help(fibonacci)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



데코레이터(trace)가 감싸고 있는 원래 함수(fibonacci)의 위치를 찾을 수 없기 때문에 <u>객체 직렬화</u>도 깨진다.

In [2]:
# wrapper함수가 local 객체이기 때문에 pickle할 수 없다는 오류가 뜬다.
import pickle

pickle.dumps(fibonacci)

AttributeError: Can't pickle local object 'trace.<locals>.wrapper'

## 문제 해결
- functools 내장 모듈에 정의된 wraps 도우미 함수를 사용한다.
- wraps함수는 데코레이터 작성을 돕는 데코레이터이다.

<br>

- wraps를 wrapper함수에 적용하면, wraps가 데코레이터 내부에 들어가는 함수에서 중요한 메타데이터를 복사해 wrapper함수해 적용해준다.

In [27]:
from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) '
              f'-> {result!r}')
        return result
    return wrapper


@trace
def fibonacci(n):
    """Return n 번째 피보나치 수""" 
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

# 데코레이터로 감싸진 함수 fibonacci에 대한 결과가 나온다.
help(fibonacci)

Help on function fibonacci in module __main__:

fibonacci(n)
    Return n 번째 피보나치 수



In [28]:
# pickle 객체 직렬화도 제대로 작동한다.
import pickle

print(pickle.dumps(fibonacci))

b'\x80\x04\x95\x1a\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\tfibonacci\x94\x93\x94.'


**객체 직렬화**: 파이썬의 객체를 일련의 바이트들로 변환한 후 나중에 다시 파이썬 객체로 복원하게 할 수 있는데, 이렇게 **파이썬 객체를 일련의 바이트들로 변환하는 것**을 직렬화(Serialization)라고 하고, 다시 바이트들을 파이썬 객체로 메모리 상에 복원하는 것을 역직렬화(Deserialization)이라 한다. 직렬화된 바이트들은 외부 장치에 저장하거나 다른 시스템으로 전송할 수 있다. 파이썬에서 객체 계층 구조를 직렬화하려면, 단순히 **dumps() 함수를 호출**하면 됩니다.

<br>출처: [예제로배우는파이썬프로그래밍_직렬화와 역직렬화](http://pythonstudy.xyz/python/article/510-%EC%A7%81%EB%A0%AC%ED%99%94%EC%99%80-%EC%97%AD%EC%A7%81%EB%A0%AC%ED%99%94)<br>
<br>참고:<br>[강의노트 04. 파이썬 pickle 모듈](https://wayhome25.github.io/cs/2017/04/04/cs-04/)
<br>   [Unit 42. 데코레이터 사용하기](https://dojang.io/mod/page/view.php?id=2427)